In [1]:
from dotenv import load_dotenv
import snowflake.connector
from fredapi import Fred
import pandas as pd
import os

load_dotenv("./.env", override=True)
FRED_API_KEY = os.getenv("FRED_API_KEY")
START_DATE = '2009-01-01'
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')
#END_DATE = (pd.to_datetime('today') - pd.Timedelta(days=1500)).strftime('%Y-%m-%d')
LIST_BRONZE_TABLES_MACROECONOMIC = ['Bronze_Consumption', 'Bronze_Investment', 'Bronze_Government_Spending', 'Bronze_Exports', 'Bronze_Imports', 'Bronze_Unemployed', 'Bronze_Labor_Force', 'Bronze_CPI', 'Bronze_Current_Account_Balance', 'Bronze_Public_Debt', 'Bronze_Interest_Rate', 'Bronze_FDI', 'Bronze_Labor_Force_Participation']
Table_Macro_Series_Mapping  = {
    'Bronze_Consumption': 'PCE',                     # Personal Consumption Expenditures (C)
    'Bronze_Investment': 'GPDIC1',                   # Gross Private Domestic Investment (I)
    'Bronze_Government_Spending': 'GCEC1',           # Government Consumption Expenditures (G)
    'Bronze_Exports': 'EXPGSC1',                     # Exports of Goods and Services (X)
    'Bronze_Imports': 'IMPGSC1',                     # Imports of Goods and Services (M)
    'Bronze_Unemployed': 'UNEMPLOY',                 # Number of Unemployed
    'Bronze_Labor_Force': 'CLF16OV',                 # Civilian Labor Force
    'Bronze_CPI': 'CPIAUCSL',                        # CPI for All Urban Consumers
    'Bronze_Current_Account_Balance': 'IEABCA',      # Current Account Balance
    'Bronze_Public_Debt': 'GFDEBTN',                 # Federal Debt: Total Public Debt
    'Bronze_Interest_Rate': 'DGS10',                 # Treasury Constant Maturity Rate
    'Bronze_FDI': 'ROWFDIQ027S',                     # Net FDI Flows
    'Bronze_Labor_Force_Participation': 'CIVPART'    # Civilian Labor Force Participation Rate
}

LIST_BRONZE_TABLES_MICROECONOMIC = ['Bronze_Micro_Household_Spending','Bronze_Micro_Small_Business_Loans', 'Bronze_Micro_Retail_Sales', 'Bronze_Micro_Personal_Income']
Table_Micro_Series_Mapping  = {
    'Bronze_Micro_Household_Spending': 'DPCERA3M086SBEA',       # Household Spending
    'Bronze_Micro_Small_Business_Loans': 'BUSLOANS',            # Small Business Loans
    'Bronze_Micro_Retail_Sales': 'RSAFS',                        # Retail Sales
    'Bronze_Micro_Personal_Income': 'PI'                        # Personal Income
}

def retrieve_data():
    manager = DataRetriever(
        api_key=FRED_API_KEY,
        start_date = START_DATE,
        end_date = END_DATE
    )
    df_micro = manager.retrieve_microdata_fred()
    df_macro = manager.retrieve_macrodata_fred()
    return df_macro, df_micro

class DataRetriever:
    def __init__(self, api_key, start_date, end_date):
        """Constructor for the DataRetriever class
        Args:
            api_key (str): FRED API key
            start_date (str): Start date for the data retrieval
            end_date (str): end date for the data retrieval
        """
        self.api_key = api_key
        self.fred = Fred(api_key=self.api_key)
        self.start_date = START_DATE
        self.end_date = END_DATE

    def retrieve_microdata_fred(self):
        fred = Fred(api_key=FRED_API_KEY)        
        for table_name in LIST_BRONZE_TABLES_MICROECONOMIC:
            # Get the corresponding FRED series based on the table name
            if table_name in Table_Micro_Series_Mapping:
                series_code = Table_Micro_Series_Mapping[table_name]
                # Fetch data from FRED using the appropriate series code and date range
                data = fred.get_series(series_code, observation_start=self.start_date, observation_end=self.end_date)
                Table_Micro_Series_Mapping[table_name] = data
            else:
                print(f"No matching FRED series found for table {table_name}")

        household_spending = Table_Micro_Series_Mapping['Bronze_Micro_Household_Spending'].resample('M').last()
        small_bussiness_loans = Table_Micro_Series_Mapping['Bronze_Micro_Small_Business_Loans'].resample('M').last()
        retails_sales = Table_Micro_Series_Mapping['Bronze_Micro_Retail_Sales'].resample('M').last()
        personal_income = Table_Micro_Series_Mapping['Bronze_Micro_Personal_Income'].resample('M').last()

        # Concatenate all series into a single DataFrame, aligning them by the common index (date)
        df = pd.concat([household_spending, small_bussiness_loans, retails_sales, personal_income], axis=1, join='inner')
        # Assign column names to the DataFrame
        df.columns = ['Bronze_Micro_Household_Spending', 'Bronze_Micro_Small_Business_Loans', 'Bronze_Micro_Retail_Sales', 'Bronze_Micro_Personal_Income']
        df.ffill(inplace=True)
        df['DateTime'] = df.index
        df['Date'] = pd.to_datetime(df['DateTime']).dt.date
        df = df.drop(columns=['DateTime'])
        df = df.rename(columns={'Date': 'DateTime'})
        # Fill any missing values with forward fill
        df.ffill(inplace=True)
        df = df.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)
        return df
    
    def retrieve_macrodata_fred(self):
        fred = Fred(api_key=FRED_API_KEY)
        for table_name in LIST_BRONZE_TABLES_MACROECONOMIC:
            # Get the corresponding FRED series based on the table name
            if table_name in Table_Macro_Series_Mapping:
                series_code = Table_Macro_Series_Mapping[table_name]
                # Fetch data from FRED using the appropriate series code and date range
                data = fred.get_series(series_code, observation_start=self.start_date, observation_end=self.end_date)
                Table_Macro_Series_Mapping[table_name] = data
            else:
                print(f"No matching FRED series found for table {table_name}")
        
        consumption = Table_Macro_Series_Mapping['Bronze_Consumption'].resample('M').last()
        investment = Table_Macro_Series_Mapping['Bronze_Investment'].resample('M').last()
        government_spending = Table_Macro_Series_Mapping['Bronze_Government_Spending'].resample('M').last()
        exports = Table_Macro_Series_Mapping['Bronze_Exports'].resample('M').last()
        imports = Table_Macro_Series_Mapping['Bronze_Imports'].resample('M').last()
        unemployed = Table_Macro_Series_Mapping['Bronze_Unemployed'].resample('M').last()
        labor_force = Table_Macro_Series_Mapping['Bronze_Labor_Force'].resample('M').last()
        cpi_series = Table_Macro_Series_Mapping['Bronze_CPI'].resample('M').last()
        current_account_balance = Table_Macro_Series_Mapping['Bronze_Current_Account_Balance'].resample('M').last()
        public_debt = Table_Macro_Series_Mapping['Bronze_Public_Debt'].resample('M').last()
        interest_rate = Table_Macro_Series_Mapping['Bronze_Interest_Rate'].resample('M').last()
        fdi = Table_Macro_Series_Mapping['Bronze_FDI'].resample('M').last()
        labor_force_participation = Table_Macro_Series_Mapping['Bronze_Labor_Force_Participation'].resample('M').last()
        
        # Concatenate all series into a single DataFrame, aligning them by the common index (date)
        df = pd.concat([consumption, investment, government_spending, exports, imports, unemployed, labor_force,
                        cpi_series, current_account_balance, public_debt, interest_rate, fdi, labor_force_participation], axis=1, 
                        join='inner')
        # Assign column names to the DataFrame
        df.columns = ['Consumption', 'Investment', 'Government_Spending', 'Exports', 'Imports', 
                    'Unemployed', 'Labor_Force', 'CPI', 'Current_Account_Balance', 'Public_Debt', 
                    'Interest_Rate', 'FDI', 'Labor_Force_Participation']
        
        # Calculate GDP using the formula GDP = C + I + G + (X - M)
        df['GDP'] = df['Consumption'] + df['Investment'] + df['Government_Spending'] + (df['Exports'] - df['Imports'])
        df.ffill(inplace=True)
        # Calculate the GDP Growth Rate
        df['GDP_Growth_Rate'] = df['GDP'].pct_change(fill_method=None) * 100
        # Calculate Inflation Rate using the CPI
        df['Inflation_Rate'] = df['CPI'].pct_change(fill_method=None) * 100
        # Calculate Unemployment Rate
        df['Unemployment_Rate'] = (df['Unemployed'] / df['Labor_Force']) * 100
        df['DateTime'] = df.index
        df['Date'] = pd.to_datetime(df['DateTime']).dt.date
        df = df.drop(columns=['DateTime'])
        df = df.rename(columns={'Date': 'DateTime'})
        # Fill any missing values with forward fill
        df.ffill(inplace=True)
        df = df.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)
        return df

df_macro_eco, df_micro_eco = retrieve_data()
#df_macro_eco = retrieve_data()

In [2]:
df_micro_eco

,Bronze_Micro_Household_Spending,Bronze_Micro_Small_Business_Loans,Bronze_Micro_Retail_Sales,Bronze_Micro_Personal_Income,DateTime
2009-01-31,84.092,1544.9018,336929.0,12125.6,2009-01-31
2009-02-28,83.747,1532.8201,335576.0,12007.4,2009-02-28
2009-03-31,83.401,1511.3496,329747.0,11965.3,2009-03-31
2009-04-30,83.277,1491.3671,331352.0,12009.4,2009-04-30
2009-05-31,83.353,1469.3714,334256.0,12179.6,2009-05-31
...,...,...,...,...,...
2024-05-31,120.279,2755.8612,704309.0,24676.2,2024-05-31
2024-06-30,120.443,2768.3212,702350.0,24738.0,2024-06-30
2024-07-31,120.949,2763.1216,710851.0,24819.3,2024-07-31
2024-08-31,121.149,2767.8128,711291.0,24876.3,2024-08-31


In [3]:
max_date = df_macro_eco['DateTime'].max()
filtered_df = df_micro_eco[df_micro_eco['DateTime'] >= max_date]
filtered_df

,Bronze_Micro_Household_Spending,Bronze_Micro_Small_Business_Loans,Bronze_Micro_Retail_Sales,Bronze_Micro_Personal_Income,DateTime
2023-01-31,116.810,2809.9671,693826.0,22855.1,2023-01-31
2023-02-28,116.785,2801.0657,686434.0,22984.2,2023-02-28
2023-03-31,116.506,2791.9343,679067.0,23104.4,2023-03-31
2023-04-30,116.900,2769.5787,683698.0,23194.0,2023-04-30
2023-05-31,116.971,2763.5493,686672.0,23299.8,2023-05-31
2023-06-30,117.095,2757.9886,688810.0,23372.5,2023-06-30
2023-07-31,117.614,2754.8967,690641.0,23427.8,2023-07-31
2023-08-31,117.637,2756.8846,696238.0,23533.1,2023-08-31
2023-09-30,117.931,2760.0446,702160.0,23636.2,2023-09-30
2023-10-31,118.174,2770.4135,699977.0,23708.3,2023-10-31


In [4]:
df_combined = pd.merge(df_macro_eco, df_micro_eco[['Bronze_Micro_Household_Spending', 'Bronze_Micro_Small_Business_Loans', 'Bronze_Micro_Retail_Sales', 'Bronze_Micro_Personal_Income']], left_index=True, right_index=True)
df_combined

,Consumption,Investment,Government_Spending,Exports,Imports,Unemployed,Labor_Force,CPI,Current_Account_Balance,Public_Debt,...,Labor_Force_Participation,GDP,GDP_Growth_Rate,Inflation_Rate,Unemployment_Rate,DateTime,Bronze_Micro_Household_Spending,Bronze_Micro_Small_Business_Loans,Bronze_Micro_Retail_Sales,Bronze_Micro_Personal_Income
2009-01-31,9847.2,2073.135,3498.103,1631.257,2028.740,12058.0,154210.0,211.933,-379729.0,11126941.0,...,65.7,15020.955,NaN,NaN,7.819208,2009-01-31,84.092,1544.9018,336929.0,12125.6
2009-02-28,9824.5,2073.135,3498.103,1631.257,2028.740,12898.0,154538.0,212.705,-379729.0,11126941.0,...,65.8,15020.955,0.000000,0.364266,8.346167,2009-02-28,83.747,1532.8201,335576.0,12007.4
2009-03-31,9773.2,2073.135,3498.103,1631.257,2028.740,13426.0,154133.0,212.495,-379729.0,11126941.0,...,65.6,15020.955,0.000000,-0.098728,8.710659,2009-03-31,83.401,1511.3496,329747.0,11965.3
2009-04-30,9772.5,1952.641,3549.027,1641.991,1946.352,13853.0,154509.0,212.709,-379729.0,11545275.0,...,65.7,14969.807,-0.340511,0.100708,8.965821,2009-04-30,83.277,1491.3671,331352.0,12009.4
2009-05-31,9791.6,1952.641,3549.027,1641.991,1946.352,14499.0,154747.0,213.022,-379729.0,11545275.0,...,65.7,14969.807,0.000000,0.147149,9.369487,2009-05-31,83.353,1469.3714,334256.0,12179.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,17987.2,4096.015,3661.345,2517.458,3495.702,5755.0,164649.0,296.341,-1012098.0,30928912.0,...,62.3,24523.116,0.000000,0.383457,3.495314,2022-09-30,115.305,2749.6517,673312.0,22431.8
2022-10-31,18096.2,4153.817,3710.100,2510.250,3455.513,5950.0,164679.0,297.863,-1012098.0,31419689.0,...,62.3,25014.854,2.005202,0.513598,3.613090,2022-10-31,115.497,2773.8320,681748.0,22579.7
2022-11-30,18094.0,4153.817,3710.100,2510.250,3455.513,5956.0,164441.0,298.648,-1012098.0,31419689.0,...,62.1,25014.854,0.000000,0.263544,3.621968,2022-11-30,115.160,2792.7658,673674.0,22638.2
2022-12-31,18134.6,4153.817,3710.100,2510.250,3455.513,5698.0,164998.0,298.812,-1012098.0,31419689.0,...,62.3,25014.854,0.000000,0.054914,3.453375,2022-12-31,115.240,2802.8365,666734.0,22718.6


In [6]:
df_macro_eco.to_csv("./Macroeconomic_Dataset.csv", index=False)
df_micro_eco.to_csv("./Microeconomic_Dataset.csv", index=False)
filtered_df.to_csv("./Microeconomic_Dataset_Exclusive_Macroeconomic_Based_on_DateTime.csv")
df_combined.to_csv("./Combined_Micro_Macroeconomic_Dataset.csv")